#### Imports

In [13]:
import os
import re
import skrf as rf
import pandas as pd

#### Paths

In [14]:
script_directory = os.getcwd()#obtem diretório atual
s2ps_path = os.path.join(script_directory,'..','..','oil_data','oil_data') #cria string caminho para todos os .s2p

#### Regex = hell

In [15]:
regex = r'SAW_06_02_2024_OIL_10W40_(\d+)\.S2P'

In [16]:
def read_s2p_data(regex, path):
    dfs = []#lista para ocncatenar todos os dicts de cada medição 
    for filename in os.listdir(path):
        match = re.match(regex, filename)
        if match:
            #obtem informação do nome do arquivo
            oil_name = "10W40"
            measurement_number = int(match.group(1))
            
            #carrega arquivo
            path_to_oil = os.path.join(path, filename)
            s2p_file = rf.Network(path_to_oil)
            
            #extrai frequência e parâmetros S do objeto Network
            frequency = s2p_file.f
            s_parameters = s2p_file.s_db
            
            #Dataframe com informações do objeto Network
            individual_oil_dict = {
                'OIL NAME': oil_name,
                'MEASUREMENT NUMBER': measurement_number,
                'FREQUENCY (MHZ)':frequency/1000000,
                'S11 (db)' : s_parameters[:,0,0],
                'S21 (db)' : s_parameters[:,1,0],
                'S12 (db)' : s_parameters[:,0,1],
                'S22 (db)' : s_parameters[:,1,1],
            }
            df = pd.DataFrame.from_dict(individual_oil_dict)
            dfs.append(df)
            
    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

#### Read Dataframes

In [17]:
df_synthetic = read_s2p_data(regex, s2ps_path)

In [18]:
df_synthetic.info

<bound method DataFrame.info of       OIL NAME  MEASUREMENT NUMBER  FREQUENCY (MHZ)  S11 (db)  S21 (db)  \
0        10W40                   1       110.000000 -0.931573 -78.40726   
1        10W40                   1       110.002000 -0.931530 -76.54260   
2        10W40                   1       110.004000 -0.931797 -77.46277   
3        10W40                   1       110.006001 -0.932706 -80.72226   
4        10W40                   1       110.008001 -0.931627 -79.30692   
...        ...                 ...              ...       ...       ...   
29995    10W40                   3       129.991999 -1.301960 -68.84161   
29996    10W40                   3       129.993999 -1.302031 -69.41974   
29997    10W40                   3       129.996000 -1.300586 -68.99509   
29998    10W40                   3       129.998000 -1.302139 -67.86802   
29999    10W40                   3       130.000000 -1.302044 -66.50051   

       S12 (db)  S22 (db)  
0     -80.58380 -1.138939  
1     -82.1

In [19]:
#assegura somente óleo de girassol lido
for name in df_synthetic["OIL NAME"]:
    if name != "10W40":
        print("Diferente de 10W40")

In [20]:
#cria .csv caso precise no futuro.
path_save_csv = os.path.join(script_directory,'..','..', 'oil_data','oil_data','CSVs')
if not os.path.exists(path_save_csv):
   os.makedirs(path_save_csv)
df_synthetic.to_csv(path_save_csv + '/10W40_measurements.csv')

In [21]:
def average_measurements(dataframe):
    #agrupa valores por frequência e calcula média do que interessa
    return dataframe.groupby('FREQUENCY (MHZ)', as_index=False)['S21 (db)'].mean()

In [22]:
df_10W40_s21 = average_measurements(df_synthetic)

In [23]:
df_10W40_s21.info


<bound method DataFrame.info of       FREQUENCY (MHZ)   S21 (db)
0          110.000000 -79.685367
1          110.002000 -78.228660
2          110.004000 -79.195877
3          110.006001 -79.919220
4          110.008001 -78.381107
...               ...        ...
9995       129.991999 -68.011847
9996       129.993999 -67.884593
9997       129.996000 -67.845993
9998       129.998000 -67.323890
9999       130.000000 -66.655403

[10000 rows x 2 columns]>

In [24]:
#cria .csv caso precise no futuro.
path_save_csv_processed = os.path.join(script_directory,'..','..', 'oil_data','oil_data','CSVs','processed')
if not os.path.exists(path_save_csv_processed):
   os.makedirs(path_save_csv_processed)
df_10W40_s21.to_csv(path_save_csv_processed + '/10W40_measurements_processed.csv')